# Cálculo de Métricas de un Sitio Web

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Cargamos el csv
df = pd.read_csv('LogsDataiku.csv')

In [3]:
df.shape

(10849, 18)

A partir del dataset anterior, calcula las siguientes métricas para el periodo de tiempo registrado
- N.º de visitas → Número de visitas (sesiones que ha tenido el sitio web)
- N.º de visitantes únicos → Número de usuarios diferentes que han visitado el sitio web
- N.º medio de páginas/visitas →Para cada visita (sesión) cuántas páginas se han visitado. Media
 para todas las visitas
- Tasa de rebote → Número de visitas (sesiones) que solo han accedido a una página
- Tasa de salida para cada página → % de veces que cada página ha sido una página de salida
- Tráfico directo → Número de visitas (sesiones) que provienen de escribir la url directamente
- Tráfico de búsqueda → Número de visitas (sesiones) que provienen de pinchar en una búsqueda
- Tráfico referido → Número de visitas (sesiones) que provienen de pinchar en un enlace de otra
página


## Número de Visitas

 seleccionamos la columna 'session_id', que contiene los identificadores de sesión únicos para cada visita al sitio web. Luego, se aplica el método notna() que devuelve el número de valores que no son NaN, es decir, el número total de visitas.

In [4]:
# Calculamos el número de visitas total utilizando la cookie de utmb (session_id)
num_visitas = df['session_id'].notna().sum()
num_visitas

np.int64(10849)

## Número de visitantes únicos


Se extrae la columna 'visitor_id' del DataFrame, que contiene identificadores únicos para cada visitante del sitio web. Luego, se utiliza el método nunique() para contar el número de valores únicos en esa columna, lo que da como resultado el número total de personas visitantes únicas al sitio web.

In [5]:
# Calculamos el número de visitantes unicos utilizando la cookie utma (visitor_id)
num_visitantes_unicos = df['visitor_id'].nunique()
num_visitantes_unicos

2537

## Número medio de páginas/visita

se agrupa el DataFrame por el 'session_id' utilizando el método groupby(). Se extrae la columna de location y se realiza la media de la suma de todos los valores. lo que nos da el número medio de páginas por visita.

In [6]:
# Calculamos el número de páginas que ha visitado cada sesion unica y hacemos la media
num_paginas_por_visita = df.groupby('session_id')['location'].count().mean()
num_paginas_por_visita

np.float64(2.749366447034972)

## Tasa de rebote

 El DataFrame se agrupa por el 'session_id' con groupby(). Seleccionando la columa location podemos ver que sesiones han accedido a una página solamente.

  Después, se compara si el tamaño de cada grupo es igual a 1, lo que indica que la sesión consiste en una sola página vista.
  
  Finalmente, se calcula la media de esta serie de booleanos dividiendo entre el numero total de sesiones unicas, lo que nos da la proporción de sesiones que resultaron en una tasa de rebote.

In [7]:
# Tasa de salida por página
rebote_count = df.groupby("session_id")["location"].count() # Que sesiones han accedido a una pagina unicamente
tasa_rebote = (rebote_count[rebote_count == 1].count() / df["session_id"].nunique())
tasa_rebote

np.float64(0.5486568677141409)

## Tasa de salida para cada página

Se filtran las filas del DataFrame donde la columna 'referer' es nula, lo que indica que es una página de salida. Luego, se agrupa este subconjunto de datos por la ubicación de la página ('location') utilizando el método groupby().

Se calcula el tamaño de cada grupo, que representa la cantidad de veces que cada página ha sido una página de salida, utilizando el método size().

 Esto se divide por el tamaño de cada grupo del DataFrame original, que representa el número total de veces que cada página ha sido visitada.

 El resultado es una serie que contiene la tasa de salida para cada página.

In [12]:
exit_pages = df[df['referer'].isna()].groupby('location').size() / df.groupby('location').size()
print("Tasa de salida para cada página:")
print(exit_pages)

Tasa de salida para cada página:
location
http://dataiku.com/                             0.483791
http://dataiku.com/applications/                0.006803
http://dataiku.com/applications/advertising/         NaN
http://dataiku.com/applications/ecommerce/      0.020833
http://dataiku.com/applications/freemium/       0.019608
                                                  ...   
http://www.dataiku.com/products/pricing/        0.050473
http://www.dataiku.com/products/shaker/         0.013245
http://www.dataiku.com/products/sharing/        0.016304
http://www.dataiku.com/products/thankyou/       0.730769
http://www.dataiku.com/products/trynow/         0.008929
Length: 96, dtype: float64


## Tráfico directo

Seleccinamos las filas del DataFrame donde la columna 'referer' es nula, lo que indica que el tráfico proviene de un acceso directo (es decir, el usuario escribió la URL directamente en la barra de direcciones del navegador).

Luego, se seleccionan los identificadores únicos de sesión ('session_id') de estas filas.

 Finalmente, se calcula el número de sesiones únicas utilizando el método nunique(), lo que nos da el número total de visitas únicas que provienen de tráfico directo.

In [9]:
# Tráfico directo → Número de visitas (sesiones) que provienen de escribir la url directamente
# es decir, donde referer sea NaN
num_trafico_directo = df[df['referer'].isna()]['session_id'].nunique()
num_trafico_directo

1098

## Tráfico de búsqueda

Seleccionamos las filas del DataFrame donde la columna 'referer' contiene referencias a motores de búsqueda como Google, Bing o Yahoo. Esto se hace mediante el método str.contains() que busca patrones de texto que contengan las cadenas de texto 'google', 'bing' o 'yahoo'.

Luego, se seleccionan los identificadores únicos de sesión ('session_id') de estas filas.

 Finalmente, se calcula el número de sesiones únicas utilizando el método nunique(), lo que nos da el número total de visitas únicas que provienen de tráfico de búsqueda.

In [10]:
# Tráfico de búsqueda → Número de visitas (sesiones) que provienen de pinchar en una búsqueda
search_traffic = df[df['referer'].str.contains('google|bing|yahoo', case=False, na=False)]['session_id'].nunique()
print("Tráfico de búsqueda:", search_traffic)

Tráfico de búsqueda: 1448


## Tráfico referido

En este fragmento, se seleccionan las filas del DataFrame donde la columna 'referer' no es nula, lo que indica que el tráfico proviene de un enlace de referencia de otra página web. Se utiliza la condición notna() para seleccionar solo aquellas filas donde 'referer' tiene un valor válido.

 Luego, se seleccionan los identificadores únicos de sesión ('session_id') de estas filas utilizando la notación de acceso a columnas.

 Finalmente, se calcula el número de sesiones únicas utilizando el método nunique(), lo que nos da el número total de visitas únicas que provienen de tráfico referido.

In [11]:
# - Tráfico referido → Número de visitas (sesiones) que provienen de pinchar en un enlace de otra página
# es decir, donde referer no sea NaN
num_trafico_referido = df[df['referer'].notna()]['session_id'].nunique()
num_trafico_referido

3244